### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [207]:
import requests

def name_cur():
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']

    max_cur = 0
    name_cur = ''

    for i in r:
        if max_cur < r[i]['Value']:
            max_cur = r[i]['Value']
            name_cur = r[i]['Name']
        
    ## вариант с List comprehension - лаконично, но два раза проходит цикл
    ## max_cur = max([r[i]['Value'] for i in r])
    ## name_cur = ''.join([r[i]['Name'] for i in r if r[i]['Value']==a])
    
    return print('валюта {} имеет максимальное значение курса = {}'.format(name_cur, max_cur))

In [208]:
name_cur()

валюта СДР (специальные права заимствования) имеет максимальное значение курса = 106.752


### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, 
а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [255]:
# coding: utf-8
import requests

def my_sum(x, y):
    return x+y

class Rate:
    def __init__(self, format='value'):
        self.format = format
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency, diff=False):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]

            # также добавил плюсом данную возможность (diff) и в общий метод вывода
            # теперь можно запрашивать  вывод не только как full, value но и diff
            # сейчас и общий вывод и "детализированные" методы работаю с diff
            if self.format == 'diff' or diff == True:
                return response[currency]['Value'] - response[currency]['Previous']
            
            if self.format == 'value':
                return response[currency]['Value']

        return 'Error'
    
    # добавил в методы курсов валют параметр diff, по умолчанию false - чтобы работало как и ранее
    # теперь при параметре diff = True или просто True выводится разница
    def eur(self, diff=False):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR', diff)
    
    def usd(self, diff=False):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD', diff)

    def AZN(self, diff=False):
        """Возвращает курс азербайджанского маната на сегодня в формате self.format"""
        return self.make_format('AZN', diff)

In [256]:
r = Rate()

In [257]:
r.eur(diff=True)

0.11769999999999925

In [258]:
Rate('diff').make_format('USD')

0.011600000000001387

### Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. 
Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. 
Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [264]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [319]:
class Designer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def international_award(self):
        # не придумал ничего лучше как отслеживать изменение целой части от деления на 7
        # буду признателен если подскажите более лаконичное решение
        prev_seniority = self.seniority
        # для каждой премии увеличиваем счетчик на 2
        self.seniority += 2
        
        # условие повышения сотрудника из задания:
        # если при делении на 7 меняется целая часть (+1) то это повышение
        if prev_seniority//7 +1 == self.seniority//7:
            self.grade_up()

        # публикация результатов
        return self.publish_grade()

In [320]:
# проверяем как работает система повышения сотрудников на примере отдела дизайнеров

# дизайнер Александр только что пришел в компанию, но по условиям задачи уже имеет 2 премии
alex = Designer('Александр', 2)

In [321]:
for i in range(20):
    alex.international_award()

Александр 1
Александр 1
Александр 2
Александр 2
Александр 2
Александр 3
Александр 3
Александр 3
Александр 3
Александр 4
Александр 4
Александр 4
Александр 5
Александр 5
Александр 5
Александр 5
Александр 6
Александр 6
Александр 6
Александр 7
